In [1]:
original = 'livestalk_2-6'
new = 'livestalk-carbondale-raw-5'

In [2]:
from shutil import copyfile
import os
import glob
import pandas as pd

In [3]:
def make_folder(path):
    if not os.path.exists(path):
        os.mkdir(path)
        os.mkdir(os.path.join(path,'train'))
        os.mkdir(os.path.join(path,'train','images'))
        os.mkdir(os.path.join(path,'train','labels'))

        os.mkdir(os.path.join(path,'valid'))
        os.mkdir(os.path.join(path,'valid','images'))
        os.mkdir(os.path.join(path,'valid','labels'))
        
        os.mkdir(os.path.join(path,'test'))
        os.mkdir(os.path.join(path,'test','images'))
        os.mkdir(os.path.join(path,'test','labels'))        
        
        print(f'folders created: {path}')
        
    else:
        print(f'a folder already exists here: {path}')

### Identify folder for original and resplit

In [4]:
orig_path = os.path.join(os.getcwd(),original)
new_path = os.path.join(os.getcwd(),new)
make_folder(new_path)

yaml_txt = """names:
- cow
nc: 1
train: {0}/train/images
val: {0}/valid/images
test: {0}/test/images
""".format(new_path)

with open(os.path.join(new_path,'data.yaml'),'w') as f:
    f.write(yaml_txt)



train_path = os.path.join(orig_path,'train')
valid_path = os.path.join(orig_path,'valid')
test_path = os.path.join(orig_path,'test')

folders created: /home/omar/datasci/w251-DL/livestalk/livestalk-carbondale-raw-5


### Get counts in old folder

In [5]:
##### Splitting without randomization
train_images = glob.glob(os.path.join(orig_path,'train','images','*.jpg'))
train_labels = glob.glob(os.path.join(orig_path,'train','labels','*.txt'))

valid_images = glob.glob(os.path.join(orig_path,'valid','images','*.jpg'))
valid_labels = glob.glob(os.path.join(orig_path,'valid','labels','*.txt'))

test_images = glob.glob(os.path.join(orig_path,'test','images','*.jpg'))
test_labels = glob.glob(os.path.join(orig_path,'test','labels','*.txt'))


all_images = train_images + valid_images + test_images
all_labels = train_labels + valid_labels + test_labels

all_images = sorted(all_images)
all_labels = sorted(all_labels)

assert len(all_images) == len(all_labels)

print('Counts in old location:')
print(f'Training: {len(train_images)}')
print(f'Validation: {len(valid_images)}')
print(f'Test: {len(test_images)}')
print(f'Total number of images: {len(all_images)}')


Counts in old location:
Training: 168
Validation: 66
Test: 5
Total number of images: 239


### Copy files to new location

In [6]:

def copy_files(images,labels,new_path,percent_train=0.8):
    assert len(images) == len(labels)
    
    images = sorted(images)
    labels = sorted(labels)
    
    num_train = round(len(images)*percent_train)
    count = 0
    
    for image,label in zip(images,labels):
        if count < num_train:
            location = 'train'
        else:
            location = 'valid'
        
        count += 1
        
        image_file = image.rsplit('/',1)[-1].rsplit('.',1)[0]
        label_file = label.rsplit('/',1)[-1].rsplit('.',1)[0]
    
        assert image_file == label_file, 'lists not aligned'
    
        new_image_path = os.path.join(new_path,location,'images',image.rsplit('/',1)[-1])
        new_label_path = os.path.join(new_path,location,'labels',label.rsplit('/',1)[-1])
        
        copyfile(image,new_image_path)
        copyfile(label,new_label_path)
    
        #if valid, copy to test as well
        if location == 'valid':
            test_image_path = os.path.join(new_path,'test','images',image.rsplit('/',1)[-1])
            test_label_path = os.path.join(new_path,'test','labels',label.rsplit('/',1)[-1])        
            copyfile(image,test_image_path)
            copyfile(label,test_label_path)
                                      
copy_files(all_images,all_labels,new_path)


### Get counts in new location

In [7]:
new_train = glob.glob(os.path.join(new_path,'train','images','*.jpg'))
new_valid = glob.glob(os.path.join(new_path,'valid','images','*.jpg'))
new_test = glob.glob(os.path.join(new_path,'test','images','*.jpg'))

new_train_labels = glob.glob(os.path.join(new_path,'train','labels','*.txt'))
new_valid_labels = glob.glob(os.path.join(new_path,'valid','labels','*.txt'))
new_test_labels = glob.glob(os.path.join(new_path,'test','labels','*.txt'))

print('Totals')
print(f'Number of training images in new location: {len(new_train)}')
print(f'Number of validation images in new location: {len(new_valid)}')
print(f'Number of test images in new location: {len(new_test)}')

Totals
Number of training images in new location: 191
Number of validation images in new location: 48
Number of test images in new location: 48
